### **Import Libraries**

In [ ]:
import os
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

### **Mount to Google Colab**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folder_path = "/content/drive/My Drive/CD_Dataset"

### **Create subdirectories for training and testing Data**

In [ ]:
import os

data_dir = "/content/drive/MyDrive/CD_Dataset/"
categories = ["cats", "dogs"]
train_test_split_ratio = 0.8  # Adjust as needed

for category in categories:
    path = os.path.join(data_dir, category)

    # Create subdirectories for training and testing data
    train_dir = os.path.join(data_dir, "train", category)
    test_dir = os.path.join(data_dir, "test", category)

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # List all image files in the category folder
    image_files = [f for f in os.listdir(path) if f.endswith(".jpg")]
    num_images = len(image_files)

    # Determine the number of images for training and testing
    num_train_images = int(num_images * train_test_split_ratio)

    # Move images to training and testing directories
    for i, image_file in enumerate(image_files):
        src_path = os.path.join(path, image_file)
        if i < num_train_images:
            dst_path = os.path.join(train_dir, image_file)
        else:
            dst_path = os.path.join(test_dir, image_file)
        os.rename(src_path, dst_path)

### **Build Artificial Neural Network**

In [ ]:
data_dir = "/content/drive/MyDrive/CD_Dataset/train"

def load_data(data_dir):
    categories = ["cats", "dogs"]
    data = []

    for category in categories:
        path = os.path.join(data_dir, category)
        label = categories.index(category)

        for img in os.listdir(path):  # Iterate over the subdirectory for each category
            img_path = os.path.join(path, img)
            img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load images as grayscale
            img_array = cv2.resize(img_array, (100, 100))  # Resize images to a consistent size
            data.append([img_array, label])

    random.shuffle(data)

    X = []
    y = []

    for features, label in data:
        X.append(features)
        y.append(label)

    X = np.array(X).reshape(-1, 100, 100, 1)  # Reshape the images
    X = X / 255.0  # Normalize pixel values to be between 0 and 1
    y = to_categorical(y, num_classes=2)  # One-hot encode labels

    return X, y

# Load and preprocess the data
X, y = load_data(data_dir)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### **Assign the Artificial Neural Network**

In [ ]:
model = keras.Sequential([
    layers.Flatten(input_shape=(100, 100, 1)),  # Input layer
    layers.Dense(128, activation='relu'),      # Hidden layer with 128 neurons and ReLU activation
    layers.Dropout(0.5),                        # Dropout layer to reduce overfitting
    layers.Dense(2, activation='softmax')       # Output layer with 2 units (cats and dogs) and softmax activation
])

### **Train the ANN**

**Compile the ANN**

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Cross-entropy loss for binary classification
              metrics=['accuracy'])

**Training the Set**

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
2/2 [==============================] - 0s 100ms/step - loss: 0.6113 - accuracy: 0.7255 - val_loss: 0.6431 - val_accuracy: 0.4615
Epoch 2/5
2/2 [==============================] - 0s 45ms/step - loss: 0.5992 - accuracy: 0.6275 - val_loss: 0.6495 - val_accuracy: 0.4615
Epoch 3/5
2/2 [==============================] - 0s 49ms/step - loss: 0.5909 - accuracy: 0.6275 - val_loss: 0.6414 - val_accuracy: 0.4615
Epoch 4/5
2/2 [==============================] - 0s 61ms/step - loss: 0.6026 - accuracy: 0.6863 - val_loss: 0.6350 - val_accuracy: 0.6923
Epoch 5/5
2/2 [==============================] - 0s 44ms/step - loss: 0.5672 - accuracy: 0.6667 - val_loss: 0.6365 - val_accuracy: 0.6923


In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_accuracy*100:.2f}%")

1/1 - 0s - loss: 0.7162 - accuracy: 0.5625 - 24ms/epoch - 24ms/step
Test accuracy: 56.25%


### **Predict the Result**

In [ ]:
def preprocess_image(image_path):
    img_array = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_array = cv2.resize(img_array, (100, 100))  # Resize to match the model's input size
    img_array = img_array / 255.0  # Normalize pixel values
    img_array = img_array.reshape(-1, 100, 100, 1)  # Reshape for model input
    return img_array

image_path = "/content/drive/MyDrive/cdDataset/dataset/single_prediction/cat_or_dog_2.jpg"
input_image = preprocess_image(image_path)

In [ ]:
predictions = model.predict(input_image)

1/1 [==============================] - 0s 21ms/step


In [ ]:
class_names = ["cat", "dog"]
predicted_class_index = np.argmax(predictions)
predicted_class = class_names[predicted_class_index]

# Print the predicted class
print(f"The model predicts this image is a {predicted_class}.")

The model predicts this image is a cat.
